In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
import zipfile, tempfile

In [2]:
if "snakemake" in locals():
    input_path = snakemake.input[0]
    output_path = snakemake.output[0]
else:
    input_path = "../../../../resources/madrid/census/lineas_limite.zip"
    output_path = "../../../../results/madrid/census/spatial.parquet"

In [3]:
# Load data
with tempfile.TemporaryDirectory(delete = True) as directory:
    with zipfile.ZipFile(input_path) as archive:
        archive.extractall(directory)

    source_path = "/".join([
        directory,
        "SHP_ETRS89",
        "recintos_municipales_inspire_peninbal_etrs89",
        "recintos_municipales_inspire_peninbal_etrs89.shp"
    ])

    df_spatial = gpd.read_file(source_path)

In [4]:
# Format
df_spatial["municipality_id"] = df_spatial["NATCODE"].str[-5:].astype(int)

df_spatial = df_spatial[["municipality_id", "geometry"]]

In [5]:
# Filter for Madrid
df_spatial = df_spatial[
    df_spatial["municipality_id"].astype(str).str.startswith("28")
].copy()

In [6]:
df_spatial.to_parquet(output_path)